In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [2]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews-2.csv")
Hotel_City_Reviews=pd.read_csv("Hotel_Info_Country_City.csv", usecols=['Hotel_Name','City'])
Hotel_City_Reviews.head()

,Hotel_Name,City
0,Hotel Arena,Amsterdam
1,K K Hotel George,London
2,Apex Temple Court Hotel,London
3,The Park Grand London Paddington,London
4,Monhotel Lounge SPA,Paris


In [58]:
Milan=Hotel_City_Reviews.loc[Hotel_City_Reviews['City']=='Milan']
Milan.Hotel_Name.unique()

array(['Hotel VIU Milan', 'Maison Borella',
       'Worldhotel Cristoforo Colombo', 'IH Hotels Milano Puccini',
       'Best Western Hotel City', 'Ch teau Monfort Relais Ch teaux',
       'Palazzo Parigi Hotel Grand Spa Milano', 'Hotel Lloyd',
       'Hotel Romana Residence', 'Hotel Crivi s', 'Hotel Galileo',
       'Carlyle Brera Hotel', 'Boscolo Milano Autograph Collection',
       'Mokinba Hotels King', 'Qualys Hotel Nasco',
       'Enterprise Hotel Design Boutique', 'IH Hotels Milano Ambasciatori',
       'NH Collection Milano President', 'Hotel Dieci', 'ME Milan Il Duca',
       'Hotel Principe Di Savoia', 'The Westin Palace', 'Glam Milano',
       'Excelsior Hotel Gallia Luxury Collection Hotel',
       'Rosa Grand Milano Starhotels Collezione', 'TownHouse 12',
       'Hotel Mozart', 'Hotel Dei Cavalieri',
       'Best Western Plus Hotel Galles', 'Starhotels Anderson',
       'Hotel Michelangelo', 'Hotel Mercure Milano Centro', 'NYX Milan',
       'The Yard Milano', 'Hotel Sunflo

In [59]:
Hotel_Milan = Hotel_Reviews[Hotel_Reviews['Hotel_Name'].isin(Milan.Hotel_Name.unique())]
len(Hotel_Milan)

38081

In [60]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LiverJoker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in Hotel_Milan.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in Hotel_Milan['Negative_Review'][i]) and "No Positive" in Hotel_Reviews['Positive_Review'][i]:
            continue
       elif "No Negative" in Hotel_Milan['Negative_Review'][i]:
            reviewP = Hotel_Milan['Positive_Review'][i]
       else:
            reviewN = Hotel_Milan['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [word for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [word for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [76]:
len(corpusN)

27719

In [63]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [64]:
from functools import reduce

In [77]:
bowA = []
for line in corpusN:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

27719

In [78]:
wordSet=set(bowAnew)


In [79]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [80]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)


In [81]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PositiveReviews = PosRev.iloc[0:10, :]
PositiveReviews

,0
room,0.748411
hotel,0.580814
breakfast,0.543729
nothing,0.438310
staff,0.420415
small,0.406980
could,0.382111
bit,0.377549
rooms,0.373683
one,0.356113


In [82]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidf = {x:y for x,y in tfidf.items() if y!=0}
    tfidfBowA = np.append(tfidfBowA, tfidf)


#sum(tfidfBowA[1].values())
tfidfBowA[0]

{'also': 0.13300490430620826,
 'area': 0.14201096767090646,
 'available': 0.16385901338481584,
 'beside': 0.27183544692527595,
 'busy': 0.19995059043049832,
 'construction': 0.20903708286686343,
 'despite': 0.20643876701587252,
 'got': 0.15593486110893415,
 'guests': 0.16923747262681446,
 'hotel': 0.1377376744682365,
 'lacklustre': 0.381469047882284,
 'little': 0.11928323183396795,
 'loungers': 0.3448174186073178,
 'meant': 0.24283961177029492,
 'might': 0.21736331328081215,
 'nice': 0.14778528426413545,
 'noisy': 0.13863953876218438,
 'open': 0.1555805629555717,
 'place': 0.1653619526073928,
 'pool': 0.17510683591370282,
 'pretty': 0.20471542357042005,
 'still': 0.16777567780369154,
 'taking': 0.22380577557549636,
 'view': 0.17453621282734993}

In [83]:
OneHotList=[]
for i in range(0, len(tfidfBowA), 1):
    tempList=[]
    for key in tfidfBowA[i]:
        if tfidfBowA[i][key]>.05:
               tempList.append(key)
    OneHotList.append(tempList)

In [84]:
import collections 
import mlxtend

In [85]:
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori

In [86]:
oht = OnehotTransactions()
oht_ary = oht.fit(OneHotList).transform(OneHotList)  
df = pd.DataFrame(oht_ary, columns=oht.columns_)  
df

,aa,aaabsolutely,aafter,aap,abadoneded,abailable,abandon,abandoned,abd,abdul,...,zilch,zin,zip,zippers,zise,zombies,zone,zoo,ztl,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.068798,[bit],1
1,0.146181,[breakfast],1
2,0.067030,[could],1
3,0.054764,[good],1
4,0.156680,[hotel],1
5,0.099102,[nothing],1
6,0.054115,[one],1
7,0.231321,[room],1
8,0.065010,[rooms],1
9,0.081641,[small],1
